In [1]:
import os
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Imports

In [3]:
# Brian's data
# df = pd.read_csv("../mimic-iv-3.1/ecoli_targets_w_lab_features.csv.gz", compression="gzip")
df = pd.read_csv("../mimic-iv-3.1/targets_w_lab_features_v2.csv.gz", compression="gzip")

C:\Users\brian\AppData\Local\Temp\ipykernel_6904\1290319428.py:3: DtypeWarning: Columns (28,53,56,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../mimic-iv-3.1/targets_w_lab_features_v2.csv.gz", compression="gzip")


In [4]:
# Kat's data
omr_df = pd.read_csv("../mimic-iv-3.1/df_omr_pt_ad_diag_pr.csv")

In [5]:
print(df.shape)
print(omr_df.shape)

(504738, 108)
(35529, 928)


### Merge

In [6]:
df = df.merge(omr_df, 
              on=["subject_id","charttime"], 
              how="left")

In [7]:
df.shape

(504738, 1034)

In [8]:
# Count NaN values in each of the new lab test columns
nan_counts = df.isna().sum()

# Display the result
print(nan_counts)
print(df.shape[0])

subject_id                                      0
hadm_id                                    321967
org_name                                        0
ab_name                                         0
charttime                                       0
interpretation                                  0
susceptible_flag                                0
charttime_target                                0
Blood - Hematocrit                          48598
Blood - Platelet Count                      51996
Blood - MCV                                 47240
Blood - Red Blood Cells                     41891
Blood - RDW                                 41876
Blood - MCH                                 41828
Blood - MCHC                                41826
Blood - White Blood Cells                   50783
Blood - Creatinine                          50738
Blood - Urea Nitrogen                       47044
Blood - Potassium                          121961
Blood - Sodium                             110031


### Feature Selection

In [9]:
pop = ['org_name','ab_name']

target = ['susceptible_flag']

numeric_features = [
    "Blood - Hematocrit",
    "Blood - Platelet Count",
    "Blood - MCV",
    "Blood - Red Blood Cells",
    "Blood - RDW",
    "Blood - MCH",
    "Blood - MCHC",
    "Blood - White Blood Cells",
    "Blood - Creatinine",
    "Blood - Urea Nitrogen",
#    "Blood - Potassium",
#    "Blood - Sodium",
    "Blood - Chloride",
    "Blood - Bicarbonate",
#    "Blood - Anion Gap",
    "Blood - Hemoglobin",
    'anchor_age',
    'Height (Inches)',
    'Weight (Lbs)',
    'bp_systolic',
    'bp_diastolic'
]
cat_features = [
    'Urine - Nitrite',
    'Urine - Bilirubin',
    'Urine - Yeast',
    'gender',
    'insurance',
    'language',
    'marital_status',
    'race'    
]
OHE_features = [
    '99',
'BLD001','BLD002','BLD003','BLD004','BLD005','BLD006','BLD007','BLD008','BLD009','BLD010',
'CIR001','CIR002','CIR003','CIR004','CIR005','CIR006','CIR007','CIR008','CIR009','CIR010',
'CIR011','CIR012','CIR013','CIR014','CIR015','CIR016','CIR017','CIR018','CIR019','CIR020',
'CIR021','CIR022','CIR023','CIR024','CIR025','CIR026','CIR027','CIR028','CIR029','CIR030',
'CIR031','CIR032','CIR033','CIR034','CIR035','CIR036','CIR037','CIR038','CIR039',
'DEN001','DEN002','DEN003',
'DIG001','DIG002','DIG003','DIG004','DIG005','DIG006','DIG007','DIG008','DIG009','DIG010',
'DIG011','DIG012','DIG013','DIG014','DIG015','DIG016','DIG017','DIG018','DIG019','DIG020',
'DIG021','DIG022','DIG023','DIG024','DIG025',
'EAR001','EAR002','EAR003','EAR004','EAR005','EAR006',
'END001','END002','END003','END004','END005','END006','END007','END008','END009','END010',
'END011','END012','END013','END014','END015','END016','END017',
'EXT001','EXT002','EXT003','EXT004','EXT005','EXT006','EXT007','EXT008','EXT009','EXT010',
'EXT011','EXT012','EXT013','EXT014','EXT015','EXT016','EXT017','EXT018','EXT019','EXT020',
'EXT021','EXT022','EXT023','EXT024','EXT025','EXT026','EXT027','EXT028','EXT029','EXT030',
'EYE001','EYE002','EYE003','EYE004','EYE005','EYE006','EYE007','EYE008','EYE009','EYE010',
'EYE011','EYE012',
'FAC001','FAC002','FAC003','FAC004','FAC005','FAC006','FAC008','FAC009','FAC010',
'FAC012','FAC013','FAC014','FAC015','FAC016','FAC017','FAC019','FAC020', #FAC011 missing
'FAC022','FAC023','FAC024','FAC025','FAC026','FAC027','FAC028','FAC029','FAC030', #FAC021 missing
'GEN001','GEN002','GEN003','GEN004','GEN005','GEN006','GEN007','GEN008','GEN009','GEN010',
'GEN011','GEN012','GEN013','GEN014','GEN015','GEN016','GEN017','GEN018','GEN019','GEN020',
'GEN021','GEN022','GEN023','GEN024','GEN025','GEN026',
'INF001','INF002','INF003','INF004','INF005','INF006','INF007','INF008','INF009','INF010',
'INF011','INF012',
'INJ001','INJ002','INJ003','INJ004','INJ005','INJ006','INJ007','INJ008','INJ009','INJ010',
'INJ011','INJ012','INJ013','INJ014','INJ015','INJ016','INJ017','INJ018','INJ019','INJ020',
'INJ021','INJ022','INJ023','INJ024','INJ025','INJ026','INJ027','INJ028','INJ029','INJ030',
'INJ031','INJ032','INJ033','INJ034','INJ035','INJ036','INJ037','INJ038','INJ039','INJ040',
'INJ041','INJ042','INJ043','INJ044','INJ045','INJ046','INJ047','INJ048','INJ049','INJ050',
'INJ054','INJ055','INJ056','INJ057','INJ059','INJ060','INJ061','INJ062','INJ063','INJ064',
'INJ065','INJ066','INJ067','INJ069','INJ070','INJ071','INJ072','INJ073','INJ074','INJ075','INJ076',
'MAL001','MAL002','MAL003','MAL004','MAL005','MAL006','MAL007','MAL008','MAL009','MAL010',
'MBD001','MBD002','MBD003','MBD004','MBD005','MBD006','MBD007','MBD008','MBD009','MBD010',
'MBD011','MBD012','MBD013','MBD014','MBD017','MBD018','MBD019','MBD020','MBD021','MBD022',
'MBD023','MBD024','MBD025','MBD026','MBD027','MBD028','MBD032','MBD034',
'MUS001','MUS002','MUS003','MUS004','MUS005','MUS006','MUS007','MUS008','MUS009','MUS010',
'MUS011','MUS012','MUS013','MUS014','MUS015','MUS016','MUS017','MUS020','MUS021','MUS022',
'MUS023','MUS024','MUS025','MUS026','MUS027','MUS028','MUS029','MUS030','MUS031','MUS032',
'MUS033','MUS034','MUS035','MUS036','MUS037','MUS038',
'NEO001','NEO002','NEO003','NEO004','NEO005','NEO006','NEO007','NEO008','NEO009','NEO010',
'NEO011','NEO012','NEO013','NEO014','NEO015','NEO016','NEO017','NEO018','NEO019','NEO020',
'NEO021','NEO022','NEO023','NEO024','NEO025','NEO026','NEO027','NEO028','NEO029','NEO030',
'NEO031','NEO032','NEO033','NEO034','NEO035','NEO036','NEO037','NEO038','NEO039','NEO040',
'NEO041','NEO042','NEO043','NEO044','NEO045','NEO046','NEO047','NEO048','NEO049','NEO050',
'NEO051','NEO052','NEO054','NEO056','NEO057','NEO058','NEO059','NEO060','NEO061','NEO062',
'NEO063','NEO064','NEO065','NEO066','NEO067','NEO068','NEO069','NEO070','NEO071','NEO072',
'NEO073','NEO074','NEO075','NEO078',
'NVS001','NVS002','NVS003','NVS004','NVS005','NVS006','NVS007','NVS008','NVS009','NVS010',
'NVS011','NVS012','NVS013','NVS014','NVS015','NVS016','NVS017','NVS018','NVS019','NVS020',
'NVS021','NVS022',
'PNL004','PNL006','PNL009','PNL010','PNL013','PNL015',
'PRG002','PRG003','PRG004','PRG005','PRG006','PRG007','PRG008','PRG009','PRG010','PRG011',
'PRG012','PRG013','PRG014','PRG015','PRG016','PRG017','PRG018','PRG019','PRG020','PRG021',
'PRG022','PRG023','PRG024','PRG025','PRG026','PRG027','PRG028','PRG029','PRG030',
'RSP001','RSP002','RSP003','RSP004','RSP005','RSP006','RSP007','RSP008','RSP009','RSP010',
'RSP011','RSP012','RSP013','RSP014','RSP015','RSP016','RSP017',
'SKN001','SKN002','SKN003','SKN004','SKN005','SKN006','SKN007',
'SYM001','SYM002','SYM003','SYM004','SYM005','SYM006','SYM007','SYM008','SYM009','SYM010',
'SYM011','SYM012','SYM013','SYM014','SYM015','SYM016','SYM017','SYM018','99.1',
'A01AA','A01AB','A01AC','A01AD','A02AA','A02AB','A02AC','A02AF','A02BA','A02BB','A02BC',
'A02BX','A03AA','A03AB','A03BA','A03CB','A03FA','A04AA','A04AD','A05AA','A06AA','A06AB',
'A06AD','A06AG','A06AH','A06AX','A07AA','A07AC','A07BA','A07DA','A07EA','A07EB','A07EC',
'A09AA','A09AB','A10AB','A10AC','A10AD','A10AE','A10AF','A10BA','A10BB','A10BF','A10BH',
'A10BJ','A10BK','A10BX','A11CC','A12AA','A12BA','A12CA','A12CC','A12CD','A16AX',
'B01AA','B01AB','B01AC','B01AD','B01AE','B01AF','B02AA','B02BA','B02BC','B02BD','B02BX',
'B03AB','B03AC','B03BA','B03XA','B05AA','B05BA','B05BB','B05BC','B05CA','B05CB','B05CX',
'B05XA','B05XB','B05XX','B06AA','C01AA','C01BA','C01BB','C01BC','C01BD','C01CA','C01CE',
'C01DA','C01EA','C01EB','C02AC','C02CA','C02DB','C02DC','C03AA','C03BA','C03CA','C03DA',
'C03DB','C05AA','C05AD','C05AE','C05AX','C05BA','C05BB','C07AA','C07AB','C07AG','C08CA',
'C08DA','C08DB','C09AA','C09CA','C09DX','C10AA','C10AC','D01AA','D01AC','D01AE','D01BA',
'D02AA','D04AA','D04AB','D05AX','D06AX','D06BB','D06BX','D07AA','D07AB','D07AC','D07AD',
'D07XA','D07XB','D07XC','D08AE','D10AA','D10AB','D10AD','D10AE','D10AF','D11AA','D11AC',
'D11AH','D11AX','G01AA','G01AF','G02AB','G02AD','G02CB','G02CC','G03AA','G03AB','G03AC',
'G03BA','G03CA','G03DA','G03DB','G03DC','G03FA','G03FB','G03XA','G03XC','G04BA','G04BC',
'G04BD','G04BE','G04BX','G04CA','G04CB','H01CB','H02AA','H02AB','H03AA','H04AA','H05AA',
'H05BX','J01AA','J01CA','J01CE','J01CF','J01CG','J01CR','J01DB','J01DD','J01DE','J01DH',
'J01DI','J01EA','J01EC','J01EE','J01FA','J01FF','J01GB','J01MA','J01XA','J01XB','J01XD',
'J01XX','J02AA','J02AB','J02AC','J02AX','J04AB','J04AC','J04AD','J05AB','J05AC','J05AE',
'J05AF','J05AG','J05AH','J05AJ','J05AR','J05AX','J06BA','J07AH','J07AL','J07BC','J07BD',
'J07BK','L01AA','L01AC','L01AX','L01BA','L01BB','L01BC','L01CB','L01CD','L01CE','L01DB',
'L01EA','L01EC','L01EF','L01EG','L01EL','L01EX','L01FA','L01FF','L01FX','L01XA','L01XF',
'L01XK','L01XX','L02AA','L02AB','L02AE','L02BA','L02BB','L02BG','L03AB','L03AX','L04AA',
'L04AB','L04AC','L04AD','L04AF','L04AH','L04AK','L04AX','M01AB','M01AC','M01AE','M01AH',
'M01CC','M02AA','M02AB','M03AB','M03AC','M03AX','M03BX','M04AA','M04AB','M05BA','N01AH',
'N01AX','N01BA','N01BB','N01BX','N02AA','N02AB','N02AE','N02AF','N02AJ','N02BA','N02BE',
'N02BF','N02CA','N02CC','N02CX','N03AA','N03AB','N03AF','N03AG','N03AX','N04AC','N04BA',
'N04BB','N04BC','N04BX','N05AA','N05AB','N05AD','N05AE','N05AH','N05AN','N05AX','N05BA',
'N05BB','N05CD','N05CF','N05CH','N05CM','N06AA','N06AB','N06AF','N06AX','N06BA','N06BC',
'N06DA','N06DX','N07AA','N07AB','N07AX','N07BA','N07BB','N07BC','P01AB','P01AX','P01BA',
'P01BB','P02CF','P03AC','R01AA','R01AB','R01AC','R01AD','R01AX','R01BA','R02AA','R02AB',
'R02AD','R02AX','R03AA','R03AC','R03AK','R03AL','R03BA','R03BB','R03BC','R03CC','R03DC',
'R03DX','R05CA','R05CB','R05DA','R06AA','R06AB','R06AD','R06AE','S01AA','S01AD','S01AE',
'S01BA','S01BC','S01CB','S01EA','S01EB','S01EC','S01EE','S01FA','S01FB','S01GA','S01GX',
'S01HA','S01XA','S02AA','S02BA','S02DA','S03AA','S03BA','V03AB','V03AC','V03AE','V03AF',
'V04CA','V04CC','V04CG','V04CX','V06DC','V07AB'
]

In [10]:
selected_cols = pop + target + numeric_features + cat_features + OHE_features
# Filter dataframe
df_filtered = df[selected_cols]

# Display the shape of the new dataframe
print(df_filtered.shape)

(504738, 945)


### Population Selection

In [11]:
# Drop all NAs
df_filtered2 = df_filtered.dropna()

print(df_filtered2.shape)
df_filtered2.head()

(31657, 945)


,org_name,ab_name,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Chloride,Blood - Bicarbonate,Blood - Hemoglobin,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,Urine - Nitrite,Urine - Bilirubin,Urine - Yeast,gender,insurance,language,marital_status,race,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO066,NEO067,NEO068,NEO069,NEO070,NEO071,NEO072,NEO073,NEO074,NEO075,NEO078,NVS001,NVS002,NVS003,NVS004,NVS005,NVS006,NVS007,NVS008,NVS009,NVS010,NVS011,NVS012,NVS013,NVS014,NVS015,NVS016,NVS017,NVS018,NVS019,NVS020,NVS021,NVS022,PNL004,PNL006,PNL009,PNL010,PNL013,PNL015,PRG002,PRG003,PRG004,PRG005,PRG006,PRG007,PRG008,PRG009,PRG010,PRG011,PRG012,PRG013,PRG014,PRG015,PRG016,PRG017,PRG018,PRG019,PRG020,PRG021,PRG022,PRG023,PRG024,PRG025,PRG026,PRG027,PRG028,PRG029,PRG030,RSP001,RSP002,RSP003,RSP004,RSP005,RSP006,RSP007,RSP008,RSP009,RSP010,RSP011,RSP012,RSP013,RSP014,RSP015,RSP016,RSP017,SKN001,SKN002,SKN003,SKN004,SKN005,SKN006,SKN007,SYM001,SYM002,SYM003,SY

In [12]:
# Look at before and after
df_grouped1 = df_filtered.groupby(['org_name', 'ab_name']).size().reset_index(name='count')
df_grouped2 = df_filtered2.groupby(['org_name', 'ab_name']).size().reset_index(name='count')

# Analysis
df_grouped3 = df_grouped1.merge(df_grouped2,
                                on = ['org_name','ab_name'],
                                how='left',
                                suffixes=('_starting', '_final'))

df_grouped3['difference'] = df_grouped3['count_starting'] - df_grouped3['count_final']
df_grouped3.sort_values(by = 'count_final', ascending=False, inplace=True)
df_grouped3

,org_name,ab_name,count_starting,count_final,difference
17,ESCHERICHIA COLI,MEROPENEM,27460,1635.0,25825.0
16,ESCHERICHIA COLI,GENTAMICIN,27467,1635.0,25832.0
15,ESCHERICHIA COLI,CIPROFLOXACIN,27466,1635.0,25831.0
13,ESCHERICHIA COLI,CEFTAZIDIME,27464,1635.0,25829.0
10,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,27461,1634.0,25827.0
14,ESCHERICHIA COLI,CEFTRIAXONE,27455,1634.0,25821.0
21,ESCHERICHIA COLI,TOBRAMYCIN,27464,1634.0,25830.0
11,ESCHERICHIA COLI,CEFAZOLIN,27455,1634.0,25821.0
22,ESCHERICHIA COLI,TRIMETHOPRIM/SULFA,27437,1634.0,25803.0
12,ESCHERICHIA COLI,CEFEPIME,27375,1632.0,25743.0


In [41]:
# Export results
# df_grouped3.to_excel("../mimic-iv-3.1/modelling_population_NAN.xlsx", index=False)

In [21]:
# Filter out those with less than 100
to_remove = df_grouped2[df_grouped2['count'] < 100]

# Perform an anti-join to remove these combinations
df_filtered3 = df_filtered2.merge(to_remove, on=['org_name', 'ab_name'], how='left', indicator=True)

# Keep only the rows that are not in the low-count combinations
df_filtered3 = df_filtered3[df_filtered3['_merge'] == 'left_only'].drop(columns=['_merge'])
df_filtered3.drop(columns=['count'],inplace=True)
df_filtered3.shape

(31329, 945)

In [22]:
# Check
df_filtered3.groupby(['org_name', 'ab_name']).size().reset_index(name='count')

,org_name,ab_name,count
0,ESCHERICHIA COLI,AMPICILLIN,1630
1,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,1634
2,ESCHERICHIA COLI,CEFAZOLIN,1634
3,ESCHERICHIA COLI,CEFEPIME,1632
4,ESCHERICHIA COLI,CEFTAZIDIME,1635
5,ESCHERICHIA COLI,CEFTRIAXONE,1634
6,ESCHERICHIA COLI,CIPROFLOXACIN,1635
7,ESCHERICHIA COLI,GENTAMICIN,1635
8,ESCHERICHIA COLI,MEROPENEM,1635
9,ESCHERICHIA COLI,NITROFURANTOIN,1426


In [38]:
final_combinations = df_filtered3[['org_name', 'ab_name']].drop_duplicates()
bacteria_antibiotic_pairs = list(final_combinations.itertuples(index=False, name=None))
bacteria_antibiotic_pairs

[('ESCHERICHIA COLI', 'AMPICILLIN'),
 ('ESCHERICHIA COLI', 'AMPICILLIN/SULBACTAM'),
 ('ESCHERICHIA COLI', 'CEFAZOLIN'),
 ('ESCHERICHIA COLI', 'CEFEPIME'),
 ('ESCHERICHIA COLI', 'CEFTAZIDIME'),
 ('ESCHERICHIA COLI', 'CEFTRIAXONE'),
 ('ESCHERICHIA COLI', 'CIPROFLOXACIN'),
 ('ESCHERICHIA COLI', 'GENTAMICIN'),
 ('ESCHERICHIA COLI', 'MEROPENEM'),
 ('ESCHERICHIA COLI', 'NITROFURANTOIN'),
 ('ESCHERICHIA COLI', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'TRIMETHOPRIM/SULFA'),
 ('PSEUDOMONAS AERUGINOSA', 'CEFEPIME'),
 ('PSEUDOMONAS AERUGINOSA', 'CEFTAZIDIME'),
 ('PSEUDOMONAS AERUGINOSA', 'CIPROFLOXACIN'),
 ('PSEUDOMONAS AERUGINOSA', 'GENTAMICIN'),
 ('PSEUDOMONAS AERUGINOSA', 'MEROPENEM'),
 ('PSEUDOMONAS AERUGINOSA', 'PIPERACILLIN/TAZO'),
 ('PSEUDOMONAS AERUGINOSA', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'PIPERACILLIN/TAZO'),
 ('KLEBSIELLA PNEUMONIAE', 'AMPICILLIN/SULBACTAM'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFAZOLIN'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFEPIME'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFTAZIDIME'),
 ('K

### OHE

In [23]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit and transform categorical data
encoded_array = encoder.fit_transform(df_filtered3[cat_features])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(cat_features))

# Concatenate with original dataframe (dropping old categorical columns)
df_final = pd.concat([df_filtered3.drop(columns=cat_features), encoded_df], axis=1)

# Display new dataframe shape
print(df_final.shape)

# Check new columns
df_final.head(5)


(31654, 1005)


,org_name,ab_name,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Chloride,Blood - Bicarbonate,Blood - Hemoglobin,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO066,NEO067,NEO068,NEO069,NEO070,NEO071,NEO072,NEO073,NEO074,NEO075,NEO078,NVS001,NVS002,NVS003,NVS004,NVS005,NVS006,NVS007,NVS008,NVS009,NVS010,NVS011,NVS012,NVS013,NVS014,NVS015,NVS016,NVS017,NVS018,NVS019,NVS020,NVS021,NVS022,PNL004,PNL006,PNL009,PNL010,PNL013,PNL015,PRG002,PRG003,PRG004,PRG005,PRG006,PRG007,PRG008,PRG009,PRG010,PRG011,PRG012,PRG013,PRG014,PRG015,PRG016,PRG017,PRG018,PRG019,PRG020,PRG021,PRG022,PRG023,PRG024,PRG025,PRG026,PRG027,PRG028,PRG029,PRG030,RSP001,RSP002,RSP003,RSP004,RSP005,RSP006,RSP007,RSP008,RSP009,RSP010,RSP011,RSP012,RSP013,RSP014,RSP015,RSP016,RSP017,SKN001,SKN002,SKN003,SKN004,SKN005,SKN006,SKN007,SYM001,SYM002,SYM003,SYM004,SYM005,SYM006,SYM007,SYM008,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM01

In [24]:
modeling_df = df_final.dropna()
modeling_df.shape

(31004, 1005)

In [31]:
# Optional Sample
# sample = modeling_df[modeling_df['ab_name']=='CEFTAZIDIME']
# sample_combinations = sample[['org_name', 'ab_name']].drop_duplicates()
# bacteria_antibiotic_pairs = list(sample_combinations.itertuples(index=False, name=None))
# print(bacteria_antibiotic_pairs)
# print(sample.shape)

[('ESCHERICHIA COLI', 'CEFTAZIDIME'), ('PSEUDOMONAS AERUGINOSA', 'CEFTAZIDIME'), ('KLEBSIELLA PNEUMONIAE', 'CEFTAZIDIME')]
(2707, 1005)


### Model Training and Selection

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dictionary to store trained models
trained_models = {}

# List to store results
results = []

# Initialize tqdm
progress_bar = tqdm(bacteria_antibiotic_pairs, desc="Model Training", unit="pair")

# Loop through each bacteria-antibiotic combination
for bacteria, antibiotic in progress_bar:
    progress_bar.set_description(f"Training {bacteria} - {antibiotic}")

    # Step 1: Filter the dataset for the specific bacteria-antibiotic combination
    df_subset = modeling_df[(modeling_df['org_name'] == bacteria) & (modeling_df['ab_name'] == antibiotic)]

    # Count susceptible and resistant cases (REMOVED NOT AS USEFUL)
    # susceptible_count = df_subset[df_subset['susceptible_flag'] == 1].shape[0]
    # resistant_count = df_subset[df_subset['susceptible_flag'] == 0].shape[0]
    
    # Step 2: Train-Test Split (80-20)
    X = df_subset.drop(columns=['org_name', 'ab_name', 'susceptible_flag'])
    y = df_subset['susceptible_flag']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Step 3: Grid Search with Cross-Validation for RandomForestClassifier
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 20],
        'min_samples_split': [2, 5, 10]
    }

    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=0) 
    grid_search.fit(X_train, y_train)

    # Step 4: Train the best model on the full training set
    best_params = grid_search.best_params_
    best_rf = RandomForestClassifier(**best_params, random_state=42)
    best_rf.fit(X_train, y_train)

    # Store the trained model dynamically
    model_name = f"rf_{bacteria}_{antibiotic}"
    trained_models[model_name] = best_rf

    # Step 5: Evaluate the model on the test set
    y_pred = best_rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    total_count = len(df_subset)

    # Count number of samples in test set
    test_count = len(y_test)
    actual_susceptible_count = (y_test == 1).sum()

    # Count number of predicted 1s (susceptible cases)
    predicted_susceptible_count = (y_pred == 1).sum()

    # Step 6: Store results with all required metrics
    results.append({
        'bacteria': bacteria,
        'antibiotic': antibiotic,
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        # 'total_count': len(df_subset),
        # 'susceptible_count': susceptible_count,
        'test_set_count': test_count,
        'test_set_actual_susceptible_count': actual_susceptible_count,
        'test_set_predicted_susceptible_count': predicted_susceptible_count,
        'n_estimators': best_params.get('n_estimators'),
        'max_depth': best_params.get('max_depth'),
        'min_samples_split': best_params.get('min_samples_split')
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

Training KLEBSIELLA PNEUMONIAE - TRIMETHOPRIM/SULFA: 100%|███████████████████████████| 32/32 [01:16<00:00,  2.38s/pair]


,bacteria,antibiotic,model_name,accuracy,precision,recall,f1_score,test_set_count,test_set_actual_susceptible_count,test_set_predicted_susceptible_count,n_estimators,max_depth,min_samples_split
0,ESCHERICHIA COLI,AMPICILLIN,rf_ESCHERICHIA COLI_AMPICILLIN,0.566563,0.521127,0.506849,0.513889,323,146,142,50,10,10
1,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,rf_ESCHERICHIA COLI_AMPICILLIN/SULBACTAM,0.595679,0.613139,0.870466,0.719486,324,193,274,200,10,2
2,ESCHERICHIA COLI,CEFAZOLIN,rf_ESCHERICHIA COLI_CEFAZOLIN,0.817901,0.815625,1.000000,0.898451,324,261,320,100,5,10
3,ESCHERICHIA COLI,CEFEPIME,rf_ESCHERICHIA COLI_CEFEPIME,0.907121,0.906542,1.000000,0.950980,323,291,321,100,10,2
4,ESCHERICHIA COLI,CEFTAZIDIME,rf_ESCHERICHIA COLI_CEFTAZIDIME,0.904321,0.903125,1.000000,0.949097,324,289,320,100,20,5
5,ESCHERICHIA COLI,CEFTRIAXONE,rf_ESCHERICHIA COLI_CEFTRIAXONE,0.882716,0.887147,0.992982,0.937086,324,285,319,100,10,2
6,ESCHERICHIA COLI,CIPROFLOXACIN,rf_ESCHERICHIA COLI_CIPROFLOXACIN,0.672840,0.698795,0.848780,0.766520,324,205,249,50,20,5
7,ESCHERICHIA COLI,GENTAMICIN,rf_ESCHERICHIA COLI_GENTAMICIN,0.873457,0.873457,1.000000,0.932455,324,283,324,200,10,2
8,ESCHERICHIA COLI,MEROPENEM,rf_ESCHERICHIA COLI_MEROPENEM,0.996914,0.996914,1.000000,0.998454,324,323,324,50,5,2
9,ESCHERICHIA COLI,NITROFURANTOIN,rf_ESCHERICHIA COLI_NITROFURANTOIN,0.943262,0.946619,0.996255,0.970803,282,267,281,50,20,2


In [47]:
# results_df.to_excel("../mimic-iv-3.1/modelling_results_2025_03_02.xlsx", index=False)

In [20]:
# Initialize and train
rf = RandomForestClassifier(n_estimators=200, max_depth = 10, min_samples_split = 5, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred_rf = rf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62      1470
         1.0       0.59      0.60      0.59      1350

    accuracy                           0.61      2820
   macro avg       0.61      0.61      0.61      2820
weighted avg       0.61      0.61      0.61      2820

Accuracy: 0.6071
Confusion Matrix:
[[906 564]
 [544 806]]


In [21]:
# Extract feature importances
importances = rf.feature_importances_

# Print feature importances with column names
feature_importance = sorted(zip(X_train.columns, importances), key=lambda x: x[1], reverse=True)

print("Feature Importances:")
for feature, importance in feature_importance:
    print(f"{feature}: {importance:.4f}")

Feature Importances:
Blood - RDW: 0.0200
Blood - Hemoglobin: 0.0192
Blood - Hematocrit: 0.0172
B05CB: 0.0148
Blood - Red Blood Cells: 0.0138
99.1: 0.0137
anchor_age: 0.0127
B05XA: 0.0123
Weight (Lbs): 0.0120
A12CA: 0.0117
J01XA: 0.0111
S01AA: 0.0105
bp_systolic: 0.0100
Height (Inches): 0.0099
Blood - MCH: 0.0097
A07AA: 0.0097
Blood - Platelet Count: 0.0092
INF003: 0.0089
bp_diastolic: 0.0089
V04CA: 0.0084
B05CX: 0.0083
Blood - MCV: 0.0082
Blood - Urea Nitrogen: 0.0081
A12CC: 0.0078
Blood - White Blood Cells: 0.0076
Blood - MCHC: 0.0074
J01CG: 0.0073
V04CC: 0.0073
N02AA: 0.0072
J01CR: 0.0071
V06DC: 0.0070
Blood - Chloride: 0.0065
J01CA: 0.0065
DIG017: 0.0065
Blood - Creatinine: 0.0061
GEN004: 0.0061
Blood - Bicarbonate: 0.0060
A10AC: 0.0057
A06AD: 0.0055
A12BA: 0.0053
D11AX: 0.0050
A02BC: 0.0050
A10AD: 0.0049
B05BB: 0.0049
B05BA: 0.0048
A10AB: 0.0046
A10AE: 0.0045
S02DA: 0.0044
R01AX: 0.0044
R03AC: 0.0042
N01BB: 0.0041
A12AA: 0.0041
H02AB: 0.0041
DIG004: 0.0041
G04BA: 0.0040
J01DD: 0.00